In [205]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import torch
from sklearn.model_selection import train_test_split
from sklearn import tree


## Data Acquisition
##### Flat Acquisition using csv files

In [206]:
movies = pd.read_csv('./data/movies.csv')
# genome_scores = pd.read_csv('./data/genome-scores.csv')
# genome_tags = pd.read_csv('./data/genome-tags.csv')
ratings = pd.read_csv('./data/ratings.csv')
tags = pd.read_csv('./data/tags.csv')

In [207]:
# print the first 5 rows of the movies, ratings, and tags dataframes
display(movies.head())
display(ratings.head())
display(tags.head())

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044
1,1,306,3.5,1147868817
2,1,307,5.0,1147868828
3,1,665,5.0,1147878820
4,1,899,3.5,1147868510


,userId,movieId,tag,timestamp
0,3,260,classic,1439472355
1,3,260,sci-fi,1439472256
2,4,1732,dark comedy,1573943598
3,4,1732,great dialogue,1573943604
4,4,7569,so bad it's good,1573943455


### Data Wrangling

In [208]:
grouped_ratings = ratings.groupby('movieId').agg({'rating': ['mean', 'count'], 'timestamp': ['mean']})
grouped_ratings.columns = ['mean_rating', 'rating_count', 'mean_timestamp']
grouped_ratings = grouped_ratings.astype({'mean_timestamp': 'int'})

# Merge dei dataset movies e ratings
# rimuove i film che non hanno rating
movies_ratings = pd.merge(movies, grouped_ratings, on='movieId')

# mantiene i duplicati, per rimuoverli usare .apply[set, list]
# tags.groupby('movieId')['tag'].apply(set)
new_tags = tags.groupby('movieId')['tag'].apply(set).apply(list)

df = pd.merge(movies_ratings, new_tags, on='movieId')

display(df.head())


,movieId,title,genres,mean_rating,rating_count,mean_timestamp,tag
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3.893708,57309,1153152210,"[rainy day watchlist, family, neighborhood, mi..."
1,2,Jumanji (1995),Adventure|Children|Fantasy,3.251527,24228,1122310117,"[Dynamic CGI Action, Fantasy, animals, Filmed ..."
2,3,Grumpier Old Men (1995),Comedy|Romance,3.142028,11804,980602256,"[Daryl Hannah, old, CLV, best friend, old peop..."
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,2.853547,2523,942460471,"[interracial relationship, divorce, revenge, C..."
4,5,Father of the Bride Part II (1995),Comedy,3.058434,11714,1004723013,"[Fantasy, Comedy, family, Touching, CLV, gynec..."


### Data Preparation (Data cleaning, Duplicates filtering, Data encoding)

##### Data Cleaning
The following results show that there are no NaN values in the dataset.

In [209]:
# print number of rows
print('Number of rows: ', df.shape[0])

# drop rows with missing values
df = df.dropna()

# print number of rows after dropping missing values
print('Number of rows after dropping missing values: ', df.shape[0])


Number of rows:  41875
Number of rows after dropping missing values:  41875


##### Data Encoding
Multicategorical One-Hot encoding for film genres

In [210]:
# one hot encoding for genres
# print every category in movies dataset
categories = set()
for s in df['genres'].str.split('|').values:
    categories = categories.union(set(s))

# add columns for each category
for category in categories:
    df[category] = df['genres'].str.contains(category).astype(int)

# delete genres column
df = df.drop(columns=['genres'])

display(df.head())

/var/folders/x8/p0tjyw5163s2pb2vfft_2jbc0000gn/T/ipykernel_11861/4060750238.py:9: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df[category] = df['genres'].str.contains(category).astype(int)


,movieId,title,mean_rating,rating_count,mean_timestamp,tag,Romance,Fantasy,Animation,Comedy,...,Thriller,Drama,War,Children,Sci-Fi,Documentary,Mystery,Film-Noir,IMAX,Musical
0,1,Toy Story (1995),3.893708,57309,1153152210,"[rainy day watchlist, family, neighborhood, mi...",0,1,1,1,...,0,0,0,1,0,0,0,0,0,0
1,2,Jumanji (1995),3.251527,24228,1122310117,"[Dynamic CGI Action, Fantasy, animals, Filmed ...",0,1,0,0,...,0,0,0,1,0,0,0,0,0,0
2,3,Grumpier Old Men (1995),3.142028,11804,980602256,"[Daryl Hannah, old, CLV, best friend, old peop...",1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,4,Waiting to Exhale (1995),2.853547,2523,942460471,"[interracial relationship, divorce, revenge, C...",1,0,0,1,...,0,1,0,0,0,0,0,0,0,0
4,5,Father of the Bride Part II (1995),3.058434,11714,1004723013,"[Fantasy, Comedy, family, Touching, CLV, gynec...",0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


##### Duplicates Filtering
The following results show that there are no duplicates in the dataset (as expected after merge operation).

In [211]:
df_preprocessed = df.copy()

# drop tag column TODO: find a way to use tag
df_preprocessed = df_preprocessed.drop(columns=['tag'])

# print number of rows
print('Number of rows: ', df_preprocessed.shape[0])

# drop duplicated rows
df_preprocessed = df_preprocessed.drop_duplicates()

# print number of rows after dropping missing values
print('Number of rows after dropping duplicated rows: ', df_preprocessed.shape[0])


Number of rows:  41875
Number of rows after dropping duplicated rows:  41875


## Data Visualization

In [212]:
""" # rating distribution from ratings.csv
sns.countplot(x='rating', data=ratings)
plt.show()

sns.boxplot(x='rating', data=ratings)
plt.show() """

" # rating distribution from ratings.csv\nsns.countplot(x='rating', data=ratings)\nplt.show()\n\nsns.boxplot(x='rating', data=ratings)\nplt.show() "

Rather than using discrete bins, a KDE plot smooths the observations with a Gaussian kernel, producing a continuous density estimate. This is used for continuous attributes like rating mean.

It is done to show differences after data aggregation in calculating mean for each film.

In [213]:
""" # rating distribution from df
sns.kdeplot(df['mean_rating'])
plt.xlabel('rating mean for film')
plt.title('Rating distribution in dataframe')
plt.show() """

" # rating distribution from df\nsns.kdeplot(df['mean_rating'])\nplt.xlabel('rating mean for film')\nplt.title('Rating distribution in dataframe')\nplt.show() "

Observing the distribution of all attributes for every file.

This is useful to show data trends, to reveal outliers and leverage points, to provide hints about modeling techniques to apply.


In [214]:
""" # print distribution for ratings
fig, axs = plt.subplots(3)
sns.kdeplot(ratings['timestamp'], ax=axs[0], color='r', label='timestamp')
sns.kdeplot(ratings['rating'], ax=axs[1], color='b', label='rating')
sns.kdeplot(ratings['movieId'], ax=axs[2], color='g', label='movieId')

axs[0].set_xlabel('timestamp')
axs[1].set_xlabel('rating')
axs[2].set_xlabel('movieId')
plt.show() """

" # print distribution for ratings\nfig, axs = plt.subplots(3)\nsns.kdeplot(ratings['timestamp'], ax=axs[0], color='r', label='timestamp')\nsns.kdeplot(ratings['rating'], ax=axs[1], color='b', label='rating')\nsns.kdeplot(ratings['movieId'], ax=axs[2], color='g', label='movieId')\n\naxs[0].set_xlabel('timestamp')\naxs[1].set_xlabel('rating')\naxs[2].set_xlabel('movieId')\nplt.show() "

In [215]:
# Scatter plot matrix
# Completamente inutile
# sns.pairplot(ratings)
# plt.show()

In [216]:
""" movies_to_show = 5
fig, axs = plt.subplots(movies_to_show, sharex=True, sharey=True, figsize=(10, 10))

most_rated_movies = ratings.groupby('movieId').count().sort_values('rating', ascending=False).head(movies_to_show).index

to_line_plot = []
for i in range(movies_to_show):
    tmp = ratings.where(ratings['movieId'] == most_rated_movies[i]).dropna()
    tmp['date'] = pd.to_datetime(tmp['timestamp'], unit='s')
    
    # Computing the mean for each month
    tmp = tmp.resample("M", on='date').mean()[['movieId', 'rating']].dropna()
    to_line_plot.append(tmp)
    movie_title = movies.where(movies['movieId'] == most_rated_movies[i]).dropna()['title'].values[0]
    axs[i].set_title("Movie: '" + movie_title + "'")
    

for i in range(movies_to_show):
    sns.lineplot(x='date', y='rating', ax=axs[i], data=to_line_plot[i])

fig.suptitle('Rating evolution for the 5 most rated movies')
plt.show() """

' movies_to_show = 5\nfig, axs = plt.subplots(movies_to_show, sharex=True, sharey=True, figsize=(10, 10))\n\nmost_rated_movies = ratings.groupby(\'movieId\').count().sort_values(\'rating\', ascending=False).head(movies_to_show).index\n\nto_line_plot = []\nfor i in range(movies_to_show):\n    tmp = ratings.where(ratings[\'movieId\'] == most_rated_movies[i]).dropna()\n    tmp[\'date\'] = pd.to_datetime(tmp[\'timestamp\'], unit=\'s\')\n    \n    # Computing the mean for each month\n    tmp = tmp.resample("M", on=\'date\').mean()[[\'movieId\', \'rating\']].dropna()\n    to_line_plot.append(tmp)\n    movie_title = movies.where(movies[\'movieId\'] == most_rated_movies[i]).dropna()[\'title\'].values[0]\n    axs[i].set_title("Movie: \'" + movie_title + "\'")\n    \n\nfor i in range(movies_to_show):\n    sns.lineplot(x=\'date\', y=\'rating\', ax=axs[i], data=to_line_plot[i])\n\nfig.suptitle(\'Rating evolution for the 5 most rated movies\')\nplt.show() '

In [217]:
""" # Same as above cell but the 3 movies with the highest standard deviation
movies_to_show = 3
fig, axs = plt.subplots(movies_to_show, sharex=True, sharey=True, figsize=(10, 10))

# Take movies with at least 1000 ratings
high_std_movies = ratings.where(
    ratings['movieId']
    .isin(ratings.groupby('movieId')
          .count()
          .where(ratings.groupby('movieId').count()['rating'] > 1000)
          .dropna().index)
    ).dropna()

# most_rated_movies = ratings.groupby('movieId').count().sort_values('rating', ascending=False).head(movies_to_show).index
high_std_movies = high_std_movies.groupby('movieId').std().sort_values('rating', ascending=False).head(movies_to_show).index

to_line_plot = []
for i in range(movies_to_show):
    tmp = ratings.where(ratings['movieId'] == high_std_movies[i]).dropna()
    tmp['date'] = pd.to_datetime(tmp['timestamp'], unit='s')
    
    # Computing the mean for each month
    tmp = tmp.resample("M", on='date').mean()[['movieId', 'rating']].dropna()
    to_line_plot.append(tmp)
    movie_title = movies.where(movies['movieId'] == high_std_movies[i]).dropna()['title'].values[0]
    axs[i].set_title("Movie: '" + movie_title + "'")

for i in range(movies_to_show):
    sns.lineplot(x='date', y='rating', ax=axs[i], data=to_line_plot[i])

# Print the titles of the movies
plt.show() """

' # Same as above cell but the 3 movies with the highest standard deviation\nmovies_to_show = 3\nfig, axs = plt.subplots(movies_to_show, sharex=True, sharey=True, figsize=(10, 10))\n\n# Take movies with at least 1000 ratings\nhigh_std_movies = ratings.where(\n    ratings[\'movieId\']\n    .isin(ratings.groupby(\'movieId\')\n          .count()\n          .where(ratings.groupby(\'movieId\').count()[\'rating\'] > 1000)\n          .dropna().index)\n    ).dropna()\n\n# most_rated_movies = ratings.groupby(\'movieId\').count().sort_values(\'rating\', ascending=False).head(movies_to_show).index\nhigh_std_movies = high_std_movies.groupby(\'movieId\').std().sort_values(\'rating\', ascending=False).head(movies_to_show).index\n\nto_line_plot = []\nfor i in range(movies_to_show):\n    tmp = ratings.where(ratings[\'movieId\'] == high_std_movies[i]).dropna()\n    tmp[\'date\'] = pd.to_datetime(tmp[\'timestamp\'], unit=\'s\')\n    \n    # Computing the mean for each month\n    tmp = tmp.resample("M", 

In [218]:
# TODO - compute np.unique(column_array, return_counts=True) for every attribute to visualize results
# follow the last example from this guide https://numpy.org/doc/stable/reference/generated/numpy.unique.html
 

Plot density for every attribute of dataset

In [219]:
""" # density plot for rating_count
fig, axs = plt.subplots(1,2, figsize=(10, 5))
sns.kdeplot(df['rating_count'], ax=axs[0], label='rating_count')
sns.kdeplot(df['rating_count'].where(df['rating_count'] < 50), ax=axs[1], label='rating_count < 50')

print('Number of movies with rating_count < 50: ', df.where(df['rating_count'] < 50).count()[0])
print('Number of all movies: ', df.shape[0]) """

" # density plot for rating_count\nfig, axs = plt.subplots(1,2, figsize=(10, 5))\nsns.kdeplot(df['rating_count'], ax=axs[0], label='rating_count')\nsns.kdeplot(df['rating_count'].where(df['rating_count'] < 50), ax=axs[1], label='rating_count < 50')\n\nprint('Number of movies with rating_count < 50: ', df.where(df['rating_count'] < 50).count()[0])\nprint('Number of all movies: ', df.shape[0]) "

In [220]:
# density plot for rating year

# convert timestamp to year
df['year_timestamp'] = pd.to_datetime(df['mean_timestamp'], unit='s').dt.year

# delete mean_timestamp
df = df.drop(columns=['mean_timestamp'])

""" sns.kdeplot(df['year_timestamp'])
plt.show() """

" sns.kdeplot(df['year_timestamp'])\nplt.show() "

In [221]:
""" # plot for categories
cat = list(categories)

# count the number of rows containing 1 for each category
cat_dict = {}
for category in cat:
    if category == '(no genres listed)':
        cat_dict['no genre'] = df[category].sum()
    else:
        cat_dict[category] = df[category].sum()

# plot an histogram of the number of rows for each category
fig, axs = plt.subplots(1, 1, figsize=(20, 10))
ax = sns.barplot(x=list(cat_dict.keys()), y=list(cat_dict.values()))
ax.bar_label(container=ax.containers[0], labels=list(cat_dict.keys()))
plt.xticks([])
plt.show() """


" # plot for categories\ncat = list(categories)\n\n# count the number of rows containing 1 for each category\ncat_dict = {}\nfor category in cat:\n    if category == '(no genres listed)':\n        cat_dict['no genre'] = df[category].sum()\n    else:\n        cat_dict[category] = df[category].sum()\n\n# plot an histogram of the number of rows for each category\nfig, axs = plt.subplots(1, 1, figsize=(20, 10))\nax = sns.barplot(x=list(cat_dict.keys()), y=list(cat_dict.values()))\nax.bar_label(container=ax.containers[0], labels=list(cat_dict.keys()))\nplt.xticks([])\nplt.show() "

## Data Preprocessing

In [222]:

# from title extract year
df['year'] = df['title'].str.extract('(\(\d{4}\))', expand=True)
# remove parentheses
df['year'] = df['year'].str.extract('(\d{4})', expand=True)

# remove movies with no year
df = df.dropna(subset=['year'])

# delete title column
df = df.drop(columns=['movieId', 'title'])

# convert year to int
df['year'] = df['year'].astype(int)

display(df.head())

,mean_rating,rating_count,tag,Romance,Fantasy,Animation,Comedy,Crime,Western,(no genres listed),...,War,Children,Sci-Fi,Documentary,Mystery,Film-Noir,IMAX,Musical,year_timestamp,year
0,3.893708,57309,"[rainy day watchlist, family, neighborhood, mi...",0,1,1,1,0,0,0,...,0,1,0,0,0,0,0,0,2006,1995
1,3.251527,24228,"[Dynamic CGI Action, Fantasy, animals, Filmed ...",0,1,0,0,0,0,0,...,0,1,0,0,0,0,0,0,2005,1995
2,3.142028,11804,"[Daryl Hannah, old, CLV, best friend, old peop...",1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,2001,1995
3,2.853547,2523,"[interracial relationship, divorce, revenge, C...",1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1999,1995
4,3.058434,11714,"[Fantasy, Comedy, family, Touching, CLV, gynec...",0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,2001,1995


In [223]:
""" # density plot for film year
sns.kdeplot(df['year'])
plt.show() """

" # density plot for film year\nsns.kdeplot(df['year'])\nplt.show() "

### Correlation results
The following results indicate that there is no correlation between the rating timestamp and the film year, as we thought.
There are instead correlations between genres like Children-Animation and Thriller-Horror like expected.
Other genres less important correlations are Children-Adventure, Fantasy-Adventure, Action-Adventure, Crime-Thriller and Mystery-Thriller.
There is a correlation also between something different from genres as rating_count-mean_rating.
The most interesting correlation (0.8) is mean_timestamp-movieId, which indicates that the period in which the rating was made is highly movie-dependent.


In [224]:
# convert timestamp to year
# df['year_timestamp'] = pd.to_datetime(df['mean_timestamp'], unit='s').dt.year
# year_timestamp can be removed because is less precise than timestamp seeing the correlation with movieId

# delete mean_timestamp column to avoid redundancy - TODO check if it is better to keep it
# df = df.drop(columns=['mean_timestamp'])

# compute analysis between attributes of the dataset
print(df.corr())
print(df.cov())
print(df.describe())

# follow examples from slides on Data Visualization (pages 6-8)
# TODO - write results of correlation in markdown
# TODO - show results of correlation in a graphic way
# TODO - write results of covariance in markdown
# TODO - show results of covariance in a graphic way
# TODO - write results of describe in markdown
# TODO - show results of describe in a graphic way


/var/folders/x8/p0tjyw5163s2pb2vfft_2jbc0000gn/T/ipykernel_11861/335129739.py:9: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  print(df.corr())
/var/folders/x8/p0tjyw5163s2pb2vfft_2jbc0000gn/T/ipykernel_11861/335129739.py:10: FutureWarning: The default value of numeric_only in DataFrame.cov is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  print(df.cov())


                    mean_rating  rating_count   Romance   Fantasy  Animation  \
mean_rating            1.000000      0.132640  0.043127 -0.018992   0.020720   
rating_count           0.132640      1.000000  0.026379  0.058577   0.016321   
Romance                0.043127      0.026379  1.000000  0.009709  -0.060998   
Fantasy               -0.018992      0.058577  0.009709  1.000000   0.152592   
Animation              0.020720      0.016321 -0.060998  0.152592   1.000000   
Comedy                -0.029512      0.033881  0.144264  0.015107   0.008276   
Crime                  0.015203      0.050338 -0.055434 -0.058943  -0.059662   
Western               -0.013533     -0.006786 -0.001002 -0.032380  -0.026986   
(no genres listed)    -0.001263     -0.039653 -0.078566 -0.045733  -0.044545   
Action                -0.083334      0.106123 -0.069615  0.058225   0.012667   
Horror                -0.224045     -0.020006 -0.113761  0.032483  -0.055188   
Adventure             -0.028135      0.1

In [225]:
# remove tag column
df = df.drop(columns=['tag'])

display(df.head())

,mean_rating,rating_count,Romance,Fantasy,Animation,Comedy,Crime,Western,(no genres listed),Action,...,War,Children,Sci-Fi,Documentary,Mystery,Film-Noir,IMAX,Musical,year_timestamp,year
0,3.893708,57309,0,1,1,1,0,0,0,0,...,0,1,0,0,0,0,0,0,2006,1995
1,3.251527,24228,0,1,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,2005,1995
2,3.142028,11804,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,2001,1995
3,2.853547,2523,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1999,1995
4,3.058434,11714,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,2001,1995


## Modeling

In [226]:
# Split data
df2 = df.copy()
X = df2.drop(columns=['mean_rating'])
y = df['mean_rating']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [229]:
import tensorflow as tf

# Create the model using the Keras API of Tensorflow
model = tf.keras.Sequential([
    tf.keras.layers.Dense(4, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(1)
])

# Compile the model, specifying the loss function and optimizer
model.compile(loss='mean_squared_error', metrics=['mean_squared_error'], optimizer='adam')

# Train the model on the training data
history = model.fit(X_train, y_train, epochs=100, batch_size=256, validation_split=0.2)


# Evaluate the model on the test data
loss, mse = model.evaluate(X_test, y_test, verbose=0)
print('Loss: ', loss)
print('MSE: ', mse)

display(y_test[:5])
model.predict(X_test[:5])

Epoch 1/100


/opt/anaconda3/lib/python3.9/site-packages/keras/engine/data_adapter.py:1696: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  return t[start:end]


105/105 [==============================] - 1s 3ms/step - loss: 1325.1180 - mean_squared_error: 1325.1180 - val_loss: 65.2286 - val_mean_squared_error: 65.2286
Epoch 2/100
105/105 [==============================] - 0s 2ms/step - loss: 17.1604 - mean_squared_error: 17.1604 - val_loss: 9.6623 - val_mean_squared_error: 9.6623
Epoch 3/100
105/105 [==============================] - 0s 2ms/step - loss: 9.2306 - mean_squared_error: 9.2306 - val_loss: 8.9570 - val_mean_squared_error: 8.9570
Epoch 4/100
105/105 [==============================] - 0s 2ms/step - loss: 8.6719 - mean_squared_error: 8.6719 - val_loss: 8.4232 - val_mean_squared_error: 8.4232
Epoch 5/100
105/105 [==============================] - 0s 2ms/step - loss: 8.1382 - mean_squared_error: 8.1382 - val_loss: 7.8920 - val_mean_squared_error: 7.8920
Epoch 6/100
105/105 [==============================] - 0s 2ms/step - loss: 7.6296 - mean_squared_error: 7.6296 - val_loss: 7.3809 - val_mean_squared_error: 7.3809
Epoch 7/100
105/105 [===

/var/folders/x8/p0tjyw5163s2pb2vfft_2jbc0000gn/T/ipykernel_11861/2749180253.py:22: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  display(y_test[:5])


36459    2.772727
24390    5.000000
4772     3.015625
20877    1.000000
16731    3.000000
Name: mean_rating, dtype: float64

1/1 [==============================] - 0s 70ms/step


array([[3.0979486],
       [3.0979486],
       [3.0979486],
       [3.0979486],
       [3.0979486]], dtype=float32)